In [ ]:
using CSV, DataFrames
include("controller/data_structure_controller.jl");

: 

In [114]:
# df = CSV.File("data/raw_tick_data.csv") |> DataFrame;
# df.date = DateTime.(df.date, "mm/dd/yyyy H:M:S.s");
# first(df, 5)

# df = CSV.File("../ahrom_edited.csv") |> DataFrame;
# df.date = DateTime.(df.date, "yyyy/mm/dd H:M:S.s");
# df.price = Float64.(df.price)
# df = df[!, ["date", "volume", "price"]]

Row,date,price,volume
,DateTime,Float64,Int64
1,2013-09-01T17:00:00.083,1640.25,8
2,2013-09-01T17:00:00.083,1640.25,1
3,2013-09-01T17:00:00.083,1640.25,2
4,2013-09-01T17:00:00.083,1640.25,1
5,2013-09-01T17:00:00.083,1640.25,1


# Expected Run Bars

In [127]:
include("controller/data_structure_controller.jl");

@time emaDollarRunBars = handleInput(
    "ExpectedDollarRunBars",
    Dict(),
    df,
    batchSize=1_000_000,  
);

first(emaDollarRunBars, 5)

batch 1 with size 1090990


batch 2 with size 1090990
batch 3 with size 1090990


batch 4 with size 1090990
batch 5 with size 1090990


  3.423512 seconds (17.49 M allocations: 1.248 GiB, 10.10% gc time, 20.02% compilation time)


Row,Date Time,Tick Number,Open,High,Low,Close,Cumulative Volume,Cumulative Buy Volume,Cumulative Sell Volume,Cumulative Ticks,Cumulative Dollar Value,Threshold
,DateTime,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2013-09-02T06:49:11.736,42464,1640.25,NaN,NaN,1646.5,136273.0,74921.0,61352.0,42464.0,2.2403e8,1.04587e8
2,2013-09-03T07:12:44.764,127170,1646.5,1649.0,1642.25,1643.75,262435.0,127504.0,134931.0,84706.0,4.31936e8,2.2206e8
3,2013-09-03T09:20:56.103,237128,1643.75,1650.0,1643.5,1644.75,419374.0,213894.0,205480.0,109958.0,6.90833e8,3.52362e8
4,2013-09-03T11:20:29.951,374235,1644.75,1647.75,1636.25,1636.5,544294.0,258883.0,285411.0,137107.0,8.9351e8,4.68485e8
5,2013-09-03T14:41:36.170,551175,1636.5,1639.0,1630.75,1637.75,695839.0,362446.0,333393.0,176940.0,1.13765e9,5.92613e8


In [128]:
@time tickStandardBars = handleInput(
    "TickStandardBars",
    Dict(
        :threshold => 2500.0
    ),
    df,
    batchSize=1_000_000,  
);

size(tickStandardBars)

batch 1 with size 1090990


batch 2 with size 1090990
batch 3 with size 1090990


batch 4 with size 1090990
batch 5 with size 1090990
  0.421221 seconds (16.74 M allocations: 557.705 MiB, 36.65% compilation time)


(2181, 12)

In [129]:
using DataFrames, CSV, Plotly
using Statistics
using KernelDensity
using StatsBase
using Dates

fig = plot([
        candlestick(
            x=round.(emaDollarRunBars."Date Time", Dates.Minute(30)),
            open=emaDollarRunBars."Open",
            high=emaDollarRunBars."High",
            low=emaDollarRunBars."Low",
            close=emaDollarRunBars."Close",
            name="EMA DRB Candles",
            increasing_line_color="green",
            decreasing_line_color="red",
            whiskerwidth=1,
        ),
        scatter(
            x=tickStandardBars."Date Time",
            y=tickStandardBars.Close,
            opacity=0.5,
        ),
    ],
    Layout(xaxis_rangeslider_visible=false),)


data: [
  "candlestick with fields close, decreasing, high, increasing, low, name, open, type, whiskerwidth, and x",
  "scatter with fields opacity, type, x, and y"
]

layout: "layout with fields margin, template, and xaxis"

In [5]:
imb_logrets = diff(log.(emaDollarRunBars[!, "Close"]))

imb_standard = (imb_logrets .- mean(imb_logrets)) ./ std(imb_logrets)

imb_kde = kde(imb_standard);
random_kde = kde(randn(size(emaDollarRunBars)[1]));

plot([
    scatter(x=imb_kde.x, y=imb_kde.density, name="Imbalamce Bars"),
    scatter(x=random_kde.x, y=random_kde.density, name="Normal")]
);

# dollar_acr = autocor(dollar_logrets, 1:10)
# plot(bar(x=1:10, y=dollar_acr), Layout(title="Dollar Bars AutoCorrelation"))

# imb_acr = autocor(imb_logrets, 1:10)
# plot(bar(x=1:10, y=imb_acr), Layout(title="Dollar Imbalance Bars (num_ticks_init = 100k, num_prev_bars=3) AutoCorrelation"))

In [5]:
@time expectedVolumeRunBars = handleInput(
    "ExpectedVolumeRunBars",
    Dict(),
    df,
    batchSize=1_000_000,  
)

expectedVolumeRunBars

batch 1 with size 200000


100000.0

200000.0

300000.0

400000.0

500000.0

600000.0

700000.0

800000.0

900000.0

1000000.0

1100000.0

1200000.0

1300000.0

1400000.0

1500000.0

1600000.0

1700000.0

1800000.0

1900000.0

2000000.0

  0.282974 seconds (1.19 M allocations: 45.542 MiB, 93.42% compilation time)


Row,Date Time,Tick Number,Open,High,Low,Close,Cumulative Volume,Cumulative Buy Volume,Cumulative Sell Volume,Cumulative Ticks,Cumulative Dollar Value
,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…


In [22]:
@time expectedTickRunBars = handleInput(
    "ExpectedTickRunBars",
    Dict(),
    df,
    batchSize=1_000_000,  
)

expectedTickRunBars

batch 1 with size 1090990
batch 2 with size 1090990
batch 3 with size 1090990


batch 4 with size 1090990


batch 5 with size 1090990
  0.656739 seconds (16.80 M allocations: 604.772 MiB, 13.91% gc time, 34.64% compilation time)


Row,Date Time,Tick Number,Open,High,Low,Close,Cumulative Volume,Cumulative Buy Volume,Cumulative Sell Volume,Cumulative Ticks,Cumulative Dollar Value
,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…


# Const Imbalance Bars

In [23]:
@time constDollarRunBars = handleInput(
    "ConstDollarRunBars",
    Dict(),
    df,
    batchSize=1_000_000,  
)

constDollarRunBars

batch 1 with size 1090990


batch 2 with size 1090990
batch 3 with size 1090990


batch 4 with size 1090990
batch 5 with size 1090990


  0.687797 seconds (16.74 M allocations: 601.700 MiB, 17.33% gc time, 23.91% compilation time)


Row,Date Time,Tick Number,Open,High,Low,Close,Cumulative Volume,Cumulative Buy Volume,Cumulative Sell Volume,Cumulative Ticks,Cumulative Dollar Value
,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…


In [15]:
@time constVolumeRunBars = handleInput(
    "ConstVolumeRunBars",
    Dict(),
    df,
    batchSize=1_000_000,  
)

constVolumeRunBars

batch 1 with size 1090990


batch 2 with size 1090990
batch 3 with size 1090990


batch 4 with size 1090990
batch 5 with size 1090990


  2.774707 seconds (16.76 M allocations: 1.247 GiB, 4.78% gc time, 5.44% compilation time)


Row,Date Time,Tick Number,Open,High,Low,Close,Cumulative Volume,Cumulative Buy Volume,Cumulative Sell Volume,Cumulative Ticks,Cumulative Dollar Value
,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…


In [16]:
@time constTickRunBars = handleInput(
    "FixedTickRunBars",
    Dict(),
    df,
    batchSize=1_000_000,  
)

constTickRunBars

batch 1 with size 1090990


batch 2 with size 1090990
batch 3 with size 1090990


batch 4 with size 1090990
batch 5 with size 1090990


  3.015813 seconds (16.76 M allocations: 1.202 GiB, 8.52% gc time, 10.26% compilation time)


Row,Date Time,Tick Number,Open,High,Low,Close,Cumulative Volume,Cumulative Buy Volume,Cumulative Sell Volume,Cumulative Ticks,Cumulative Dollar Value
,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…
